# Terminal Distribution Generator

### Introduction
This notebook produces risk neutral distribution functions based on the Breeden-Litzenberger function, for a given set of options, spot, and rate data.

A distribution will be found for every expiry date of every snapshot date given in the options data (i.e. multiple DTEs for each snapshot date) provided that there is sufficient data in the first place. 

### Breeden-Litzenberger
A probabilty density function $$ f_{S_T}^\mathbb{Q} (s) $$ which is the density of the terminal spot price $S_T$ (at the expiration date $T$) conditional under the risk-neutral measure $\mathbb{Q}$.

# Contents

### 1. **Importing relevant libraries and data**

* Options data from [EODHD](https://eodhd.com/)

* Spot data from [Yahoo Finance](https://sg.finance.yahoo.com/quote/QQQ/)

* Rate data from [Federal Funds Effective Rate](https://fred.stlouisfed.org/series/DFF)

### 2. **Helper functions**

* ```get_rate``` returns the FED rate for a given date

* ```filter``` returns filtered options data

* ```get_dte``` returns DTE in days 

* ```get_spot_price``` returns the spot price for a given date

* ```get_call_price``` and ```get_put_price``` return the price of a call and put respectively for a given strike

* ```mono_convex``` checks a grid of $(K, C)$ pairs for monotonocity and convexity, and returns a Boolean depending on the outcome of the tests

### 3. **Main Procedure functions:**

* ```build_grid``` returns a grid of $(K, C)$ pairs

* ```smooth``` returns a smoothened function $C(K)$ given a grid

* ```bl``` returns a tuple of (strike, RND) pairs base on Breeden-Litzenberger 

### 4. **Executing main procedure**

The current content here resembles a test and can be modified to output the RND for a certain DTE or other metadata if required.  

# 1. Importing Data

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random
from scipy.interpolate import PchipInterpolator 
from scipy.ndimage import uniform_filter1d

OPTIONS = pd.read_csv("../data/qqq_options_daily_anchors_2y.csv") ### EDIT CSV FILE PATH ACCORDINGLY ###

SPOTS = pd.read_csv("../data/qqq_daily_ohlcv_2024_2026.csv", ### EDIT CSV FILE PATH ACCORDINGLY ###
                    skiprows=3,  # Skip the 3 header rows
                    index_col=0,  # date becomes index
                    names=['date', 'close', 'high', 'low', 'open', 'volume']
)
SPOTS.index.name = 'date' # ensure index name is still date

RATES = pd.read_csv('../data/DFF.csv', ### EDIT CSV FILE PATH ACCORDINGLY ###
                    index_col=0 # First column (date) becomes index
)
RATES.index.name = 'date' # use date as index
RATES.rename(columns = {'Rate': 'rate'}, inplace = True) # rename Rate to rate

/var/folders/vx/9yhw3gfn3q51s5m5qzn_xfnm0000gn/T/ipykernel_1707/2287935764.py:9: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  OPTIONS = pd.read_csv("../data/qqq_options_daily_anchors_2y.csv") # edit csv file path accordingly


# 2. Helper Functions

## FED Rate

Returns the FED rate

In [651]:
def get_rate(rates, date):
    '''Returns FED rate for a given date'''
    for idx, row in rates.iterrows():
        if idx == date:
            return row['rate']
    return None

## Filtering options data for a given snapshot and expiration
Returns filtered options data, for a given snapshot date and expiration date, and None if the data is empty after filtering

In [652]:
def filter(options, snap, exp):
    ''' 
    Returns filtered options data
    None if data is empty
    '''
    data = options[(options['snapshot_date'] == snap) &
                   (options['exp_date'] == exp)
                   ]
    return None if data.empty else data

## DTE in days
Returns DTE in days for a given snapshot date and expiration date

In [653]:
def get_dte(snap, exp):
    '''Returns DTE in days'''
    snap = pd.to_datetime(snap)
    exp = pd.to_datetime(exp)
    dte = (exp - snap).days
    return dte

## Spot prices

Returns the spot price for a given date, and None if the date cannot be found in the spot data

In [654]:
def get_spot_price(spots, snap):
    '''
    Returns spot price
    None if price not found
    '''
    if snap not in spots.index:
        return None
    else:
        return spots.loc[snap, 'close']

## Call and Put prices
Returns the price of either a call or put based on options data that is **already filtered** using ```filter```, for a given snapshot date and expiration date.

If the filtered options data is empty, or at least one of bid and ask data is invalid, returns None.

In [655]:
def get_call_price(options, k):
    '''
    Returns call price
    None if filtered options data is empty
    None if either bid or ask is invalid
    '''
    data = options[(options['type'] == 'call') &
                   (options['strike'] == k)
                   ]
    if data.empty: # if no calls are found
        return None
    row = data.iloc[0]
    bid = row['bid']
    ask = row['ask']
    if pd.isna(bid) or pd.isna(ask) or bid <= 0 or ask <= 0 or bid >= ask: # invalid bid ask values
        return None
    return (row['bid'] + row['ask']) / 2
    
def get_put_price(options, k):
    '''
    Returns put price
    None if filtered options data is empty
    None if either bid or ask is invalid
    '''
    data = options[(options['type'] == 'put') &
                   (options['strike'] == k)
                   ]
    if data.empty: # if no puts are found
        return None
    row = data.iloc[0]
    bid = row['bid']
    ask = row['ask']
    if pd.isna(bid) or pd.isna(ask) or bid <= 0 or ask <= 0 or bid >= ask: # invalid bid ask values
        return None
    return (row['bid'] + row['ask']) / 2
    

## Monotonocity and Convexity checker
For a given grid of $(K, C)$ pairs, test whether monotonocity and convexity are maintained by temporarily adding the pair to the existing grid before removing it again.

Return True it passes all the tests, False otherwise.

In [656]:
def mono_convex(grid, k, c):
    '''
    Returns True if all tests are passed, False otherwise.
    '''
    grid.append((k, c)) # add current pair
    grid.sort(key = lambda pair: pair[0]) # sort by strike

    # position of new pair post sort
    curr_idx = next(i for i, pair in enumerate(grid) if pair == (k, c))
    
    def get_k(idx):
        return grid[idx][0]
    
    def get_c(idx):
        return grid[idx][1]
    
    if len(grid) >= 2:
        if get_c(curr_idx) >= get_c(curr_idx - 1):
            grid.remove((k, c)) # remove most recent pair
            return False
    
    if len(grid) >= 3:
        # spreads
        dc1 = get_c(curr_idx-2) - get_c(curr_idx-1)
        dc2 = get_c(curr_idx-1) - get_c(curr_idx)
        dk1 = get_k(curr_idx-1) - get_k(curr_idx-2)
        dk2 = get_k(curr_idx) - get_k(curr_idx-1)

        # normalised spreads
        norm_spread1 = dc1 / dk1
        norm_spread2 = dc2 / dk2

        # check convexity with some tolerance
        if norm_spread2 > norm_spread1 * 5: 
            grid.remove((k, c)) # remove most recent pair
            return False
        
    grid.remove((k, c)) # remove most recent pair
    return True

# 3. Main Procedure Functions

## Build $(K, C)$ grid
Returns a tuple of $(K, C)$ pairs which form the grid  for a given snapshot date and expiration date

1. Filter options data with ```filter```

2. Extract all unique strikes, then loop through to add relevant pairs to the grid

    * Skip strikes with no existing options, spot, or rate data

    * Prioritise OTM options

    * If $K > S$, use $C$. If $K > S$, use $P$. In the case where only one is available, use put-call parity to find the other if necessary.

    * Put call parity is given by $$ C_t = P_t + e^{-r(T-t)}(F_t - K) $$
    where the forward is given by $$F_t = S_t e^{(r-q)(T-t)}$$ where $q$ is the dividend and assumed to be a constant of 4.6%.

    * Ensure no-arbitrage holds

3. Add relevant pairs to the grid

4. Remove the lowest and highest 15% of strikes from the grid 


In [657]:
def build_grid(options, spots, snap, exp, dte, r):
    '''Returns a tuple of $(K, C)$ pairs which form the grid'''

    options_dte = filter(options, snap, exp)
    if options_dte is None: # empty filtered options data
        # print(f"No options data for snapshot date: {snap} and expiry date: {exp}")
        return

    t = dte / 365 # dte in trading days
    q = 0.046 # dividends

    strikes = sorted(options_dte['strike'].unique())
    grid = []

    for k in strikes:

        s = get_spot_price(spots, snap)
        c = get_call_price(options_dte, k)
        p = get_put_price(options_dte, k)
        f = s * math.exp((r - q) * t) # forward

        if s is None:
            print(f"Spot price not found on {snap}")
            continue 

        if r is None:
            print(f"Rate not found on {snap}")
            continue

        if k > s: # OTM call
            if c is None:
                if p is None:  # both call and puts do not exist
                    continue
                else: # put exists but call doesnt, use put call parity
                    final_c = p + math.exp(-r * t) * (f - k)
            else:    
                final_c = c

        else: # OTM put
            if p is None: 
                if c is None:  # both call and puts do not exist
                    continue
                else: # call exists but put doesnt, directly add call
                    final_c = c
            else: # put call parity for call price   
                c = p + math.exp(-r * t) * (f - k)
                final_c = c

        # no arbitrage: non-neg, > intrinsic value, <=s, 
        if final_c <= 0 or final_c < max(0, s - k) or final_c > s:
            continue # skip current iteration as arbitrage exists

        if mono_convex(grid, k, final_c): # pass monotonicity and convexity tests
            grid.append((k,final_c))

    if len(grid) > 1: # sort grid by strikes, remove lowest 15% and highest 15%  
        grid.sort(key=lambda x: x[0])
        min = int(len(grid) * 0.15)
        cut = int(len(grid) * 0.85) 
        grid = grid[min:cut]

    return grid

## Form $C(K)$ function with PCI
Returns a smoothened function $C(K)$ given a grid of strikes and call prices.

```PchipInterpolator``` creates a smooth curve connecting each pair in the grid, and was chosen as it retains monotonicity which is necessary for no-arbitrage.

In [658]:
def smooth(grid):     
    '''Returns smoothened function C(K)'''
    if len(grid) < 5:
        # print("Too little values to interpolate function")
        return
    strikes = np.array([pair[0] for pair in grid])
    calls = np.array([pair[1] for pair in grid])

    # ensure sorted
    idx = np.argsort(strikes)
    strikes = strikes[idx]
    calls = calls[idx]

    return PchipInterpolator(strikes, calls, extrapolate=False)

## Form the BL function $f(K)$

The Breeden-Litzenberger is explicitly given by $$ f(K) = e^{rT} \frac{\partial^2 C}{\partial K^2} $$ where $C(K)$ is the smoothened function from before.

1. Form an array of sorted and evenly spaced out strikes
    * The array will take its smallest and largest values from the market strikes of $C(K)$
    
    * Will aid in differentiating $C(K)$ in the next step.

2. Differentiate $C(K)$ to the second order
    * Floor to zero any negative values 

3. To each second order, multiply by $e^{rT}$
    * $r$ is the rate of the current snapshot date
    * $T$ is the current DTE divided by 365

4. Normalise with ```trapezoid``` 
    * Calculate area under $f(K)$ using trapezoid rule
    * Area should add to 1 by right as $f$ is a PDF
    * Divide each function by the actual area to ensure that new integral will add to 1

5. Smoothen curve with ```uniform_filter1d```
    * Replaces each point with the average of itself and some neighbours
    * Reduces number of jagged lines to smoothen curve

In [659]:
def bl(grid, spline, rate, dte):
    '''
    Returns a tuple of (strikes, RND) based on Breeden-Litzenberger
    If spline does not exist, return None
    If density cannot be normalised, return None
    '''
    if spline is None:
        return 
    
    r = rate
    t = dte / 365

    strikes = [pair[0] for pair in grid]
    strikes = np.array(strikes) # convert to array

    # linearly spaced grid for plotting and integration
    linear_strikes = np.linspace(strikes.min(), strikes.max(), 50)

    second_order = spline(linear_strikes, 2) 
    
    if np.any(second_order < 0): # floor in case of negative second order
        second_order = np.maximum(second_order, 0)
    rnd = np.exp(r * t) * second_order # bl func

    total = np.trapezoid(rnd, linear_strikes) # normalise as intgeral = 1 since BL is a PDF
    if total <= 0: # cannot normalise
        return
    
    rnd = rnd / total

    rnd = uniform_filter1d(rnd, size=5) # 5-point moving average
    total = np.trapezoid(rnd, linear_strikes) # Re-normalize after smoothing
    if total <= 0: # cannot normalise
        return
    
    rnd = rnd / total

    return (tuple(linear_strikes), tuple(rnd))


# 4. Test

Each snapshot date consists of options with different expiry times, or DTEs. Our goal is to compute the RND for each expiry date of each snapshot date to end up with an array of RNDs.

The RNDs and other metadata will be saved in a dictionary ```rnd_dict``` with structure as shown:
* Key: Tuple of (snapshot date, expiration date)
* Values: A dictionary with keys as shown and in the same order:
    * dte
    * rate
    * grid
    * spline, or $C(K)$
    * density, or $F(K)$

Adjust the number of snapshot dates to test by changing $N$.

The code below will return the RND of a randomly chosen (snapshot, expiry) pair from ```rnd_dict```.

In [ ]:
# initialising options data
snaps = sorted(OPTIONS['snapshot_date'].unique())
exps = sorted(OPTIONS['exp_date'].unique())

N = 3 # ADJUST N HERE, LARGER N WILL INCREASE RUN TIME
# commenting N away will default to using all available snapshot dates

# generate keys for dict
keys = []
for snap in snaps[:N]:
    for exp in exps:
        dte = get_dte(snap, exp)
        if dte > 64: # max dte of 64, no point looping through further exp dates
            break
        keys.append((snap, exp))

subkeys = ['dte', 'rate', 'grid', 'spline', 'density']

# initialise storage dict keys
rnd_dict = {key: {subkey: None for subkey in subkeys}
            for key in keys}        
# format: {(snap, exp): {dte:, rate:, grid:, spline:, density:}}

for snap, key in keys: # initialise dtes and rates
    rnd_dict[(snap, key)]['dte'] = get_dte(snap, exp)
    rnd_dict[(snap, key)]['rate'] = get_rate(RATES, exp)

for snap, exp in keys:
    entry = rnd_dict[(snap, exp)]

    r = entry['rate']
    dte = entry['dte']

    # building grid of strike, call pairs
    grid = build_grid(OPTIONS, SPOTS, snap, exp, dte, r)
    if grid is None:
        continue
    else:
        entry['grid'] = grid

    # forming spline
    spline = smooth(grid)
    entry['spline'] = spline

    # BL function
    density = bl(grid, spline, r, dte)
    if density is None:
        continue
    entry['density'] = density

# remove entries with None values in subdict
rnd_dict = {
    key: subdict
    for key, subdict in rnd_dict.items()
    if None not in subdict.values()
}

# random selection from dictionary
key = random.choice(list(rnd_dict.keys()))
snap, exp = key
entry = rnd_dict[key]

strikes, rnds = entry['density']
plt.plot(strikes, rnds)
plt.xlabel('Strike')
plt.ylabel('Risk-Neutral Density')
plt.title(f'Function for snapshot date {snap} to expiry date {exp}')
plt.show()
    